## Start

In [1]:
# ------------ Test Rappor Mechanism ------------------------------ #
import sys, pathlib

# Path of the folder *above* "test"
project_root = pathlib.Path().resolve().parent

# Add it (only once) to sys.path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


from internal.initialize_data import *
from internal.normalizer import *
from internal.evaluation import *
from other_solutions.Rappor import *
import os
import progressbar  # pip install progressbar

EPSILON = os.environ.get('EPSILON', 1)
RADNOM_SEED = os.environ.get('RADNOM_SEED', 10)
DATASET_NUMBER = os.environ.get('DATASET_NUMBER', 2)
LIMITED_NUMBER = os.environ.get('LIM', 100)
EVOLUTION_DOMAIN_SIZE = 360  # in order to Syn.csv
ALPHA = 0.4
epsiolon1 = ALPHA * EPSILON

## Initialize dataset

In [2]:
df = read_evolution_dataset('../dataset/Syn.csv')
_, evolution_dataset = read_dataset(f'../dataset/Data{DATASET_NUMBER}-coarse.dat', dataFrame=df, limited_number=int(LIMITED_NUMBER))
tau = len(evolution_dataset[0])
n = len(evolution_dataset)

print('number of users is', n)
print('evolution_dataset[0][:10] is',evolution_dataset[0][:10])
print('tau is', tau)

Reading dataset ...
Append evolution data ...
number of users is 100
evolution_dataset[0][:10] is [102, 56, 56, 56, 56, 145, 145, 145, 145, 145]
tau is 120


## Real frequency for each data collection $t \in [\tau]$

In [3]:
dic_real_freq = compute_frequency(evolution_dataset, tau, EVOLUTION_DOMAIN_SIZE)

## Perturbation with Rappor

In [4]:
r_test = Rappor_Class(EPSILON, epsiolon1, RADNOM_SEED)
perturbed_test, _ = r_test.RAPPOR_Client(evolution_dataset[0], EVOLUTION_DOMAIN_SIZE)

print(f'Value {evolution_dataset[0][0]} perturebed to {perturbed_test[0]}')

Value 102 perturebed to [0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 1. 0. 0.

In [5]:
rappor_obj = Rappor_Class(EPSILON, epsiolon1, RADNOM_SEED)
perturbed_evolution_dataset = []

prog = progressbar.ProgressBar(maxval=n)
prog.start()
for index, user_row in enumerate(evolution_dataset):
    perturbed_row, _ = rappor_obj.RAPPOR_Client(user_row, EVOLUTION_DOMAIN_SIZE)
    perturbed_evolution_dataset.append(perturbed_row)
    prog.update(index)
prog.finish()

100% |########################################################################|


## Evaluate Frequency Estimation

In [8]:
dic_estimate_freq = []
for t in range(tau):
    dic_estimate_freq.append(rappor_obj.RAPPOR_Aggregator(get_coloumn_dataset(perturbed_evolution_dataset, t)))

print_table(dic_real_freq[0][:10], dic_estimate_freq[0][:10], 'real frequency', 'estimate frequency')

print('MSE of frequency is', findMSE(dic_real_freq, dic_estimate_freq))

|   real frequency |   estimate frequency |
|------------------|----------------------|
|             0    |           0          |
|             0.01 |           0.00507314 |
|             0    |           0.00379957 |
|             0    |           0.00507314 |
|             0    |           0.00125243 |
|             0    |           0.00507314 |
|             0    |           0          |
|             0    |           0.00507314 |
|             0.01 |           0.00379957 |
|             0    |           0.00762028 |
MSE of frequency is 4.319497604025778e-05
